# Обогащение данных

1. Заполнить отсутствующие значения по погоде интерполяционными данными.
2. Для точки росы вычесть температуру воздуха.
3. Напрвыление ветра разложить на синус и косинус.
4. Для тем. воздуха вычеслить 1-ю и 2-ю производные.
5. Вывести параметры по праздничным дням, дням недели, месяцам и неделям года.
6. Посчитать модель линейной регрессии по первым 20 зданиям и найти точность.